In [195]:
import pandas as pd
import numpy as np

df = pd.read_csv('./csv/전체데이터.csv', encoding='cp949')
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결)
0,(주)CMG제약,58820,2000/12,NaN,NaN
1,(주)CMG제약,58820,2001/12,1.03,NaN
2,(주)CMG제약,58820,2002/12,-9.99,NaN
3,(주)CMG제약,58820,2003/12,-3.37,NaN
4,(주)CMG제약,58820,2004/12,-22.92,NaN
...,...,...,...,...,...
33501,흥아해운(주),3280,2015/12,0.79,1.02
33502,흥아해운(주),3280,2016/12,0.04,0.21
33503,흥아해운(주),3280,2017/12,-0.61,-0.42
33504,흥아해운(주),3280,2018/12,-1.26,-1.16


In [178]:
df.isna().sum()
df = df.fillna(12345678.9) # 일단 결측치 어떻게 할지 몰라 이렇게 합니다

In [179]:
for i in range(len(df)):
    if df.loc[i,'이자보상배율(이자비용)'] == 12345678.9 and df.loc[i,'이자보상배율(이자비용)(IFRS연결)'] == 12345678.9: # 두개 다 결측치 인 경우 일단 2로...
        df.loc[i, '부실징후기업'] = 2
    elif df.loc[i,'이자보상배율(이자비용)(IFRS연결)'] == 12345678.9 and df.loc[i,'이자보상배율(이자비용)'] < 1: # GAAP만 고려할 경우
        df.loc[i,'부실징후기업'] = 1
    elif df.loc[i,'이자보상배율(이자비용)(IFRS연결)'] == 12345678.9 and df.loc[i,'이자보상배율(이자비용)'] > 1: # GAAP만 고려할 경우
        df.loc[i,'부실징후기업'] = 0
    elif df.loc[i,'이자보상배율(이자비용)'] == 12345678.9 and df.loc[i,'이자보상배율(이자비용)(IFRS연결)'] > 1: # IFRS만 고려할 경우
        df.loc[i,'부실징후기업'] = 0
    elif df.loc[i,'이자보상배율(이자비용)'] == 12345678.9 and df.loc[i,'이자보상배율(이자비용)(IFRS연결)'] < 1: # IFRS만 고려할 경우
        df.loc[i,'부실징후기업'] = 1
    elif df.loc[i,'이자보상배율(이자비용)(IFRS연결)'] < 1: # 두개 다 배율이 있을 때 IFRS만 고려할 경우
        df.loc[i,'부실징후기업'] = 1
    elif df.loc[i,'이자보상배율(이자비용)(IFRS연결)'] > 1: # 두개 다 배율이 있을 때 IFRS만 고려할 경우
        df.loc[i,'부실징후기업'] = 0
df # 999999999는 일단 생각하지 않은 채로 진행합니다.

,회사명,거래소코드,회계년도,이자보상배율(이자비용),이자보상배율(이자비용)(IFRS연결),부실징후기업
0,(주)CMG제약,58820,2000/12,12345678.90,12345678.90,2.0
1,(주)CMG제약,58820,2001/12,1.03,12345678.90,0.0
2,(주)CMG제약,58820,2002/12,-9.99,12345678.90,1.0
3,(주)CMG제약,58820,2003/12,-3.37,12345678.90,1.0
4,(주)CMG제약,58820,2004/12,-22.92,12345678.90,1.0
...,...,...,...,...,...,...
33501,흥아해운(주),3280,2015/12,0.79,1.02,0.0
33502,흥아해운(주),3280,2016/12,0.04,0.21,1.0
33503,흥아해운(주),3280,2017/12,-0.61,-0.42,1.0
33504,흥아해운(주),3280,2018/12,-1.26,-1.16,1.0


In [180]:
df_1 = df[['회사명','거래소코드','회계년도','부실징후기업']]
df_1

,회사명,거래소코드,회계년도,부실징후기업
0,(주)CMG제약,58820,2000/12,2.0
1,(주)CMG제약,58820,2001/12,0.0
2,(주)CMG제약,58820,2002/12,1.0
3,(주)CMG제약,58820,2003/12,1.0
4,(주)CMG제약,58820,2004/12,1.0
...,...,...,...,...
33501,흥아해운(주),3280,2015/12,0.0
33502,흥아해운(주),3280,2016/12,1.0
33503,흥아해운(주),3280,2017/12,1.0
33504,흥아해운(주),3280,2018/12,1.0


In [201]:
sefe = []
die = []
for i in range(len(df_1)-2): # 3년 연속 부실징후가 1일 때
    if df_1.loc[i,'부실징후기업'] == 1:
        if df_1.loc[i+1,'부실징후기업'] == 1:
            if df_1.loc[i+2,'부실징후기업'] == 1:
                df_1.loc[i+2,"한계기업"] = 1
            else:
                df_1.loc[i+2,'한계기업'] = 0
        else:
            df_1.loc[i+1,'한계기업'] = 0
    else:
        df_1.loc[i,'한계기업'] = 0
df_1

,회사명,거래소코드,회계년도,부실징후기업,한계기업
0,(주)CMG제약,58820,2000/12,2.0,0.0
1,(주)CMG제약,58820,2001/12,0.0,0.0
2,(주)CMG제약,58820,2002/12,1.0,NaN
3,(주)CMG제약,58820,2003/12,1.0,NaN
4,(주)CMG제약,58820,2004/12,1.0,1.0
...,...,...,...,...,...
33501,흥아해운(주),3280,2015/12,0.0,0.0
33502,흥아해운(주),3280,2016/12,1.0,NaN
33503,흥아해운(주),3280,2017/12,1.0,NaN
33504,흥아해운(주),3280,2018/12,1.0,1.0


In [182]:
# for i in range(len(df_1)-1): # 2년 연속 부실징후가 1일 때
#     if df_1.loc[i,'부실징후기업'] == 1:
#         if df_1.loc[i+1,'부실징후기업'] == 1:
#             df_1.loc[i+1,"한계기업"] = 1
#         elif df_1.loc[i+1,'부실징후기업'] != 1:
#             df_1.loc[i+1,'한계기업'] = 0
#     else:
#         df_1.loc[i,'한계기업'] = 0
# df_1

In [183]:
q = df_1[df_1['한계기업']==0]
q

,회사명,거래소코드,회계년도,부실징후기업,한계기업
0,(주)CMG제약,58820,2000/12,2.0,0.0
1,(주)CMG제약,58820,2001/12,0.0,0.0
7,(주)CMG제약,58820,2007/12,0.0,0.0
11,(주)CMG제약,58820,2011/12,0.0,0.0
14,(주)CMG제약,58820,2014/12,0.0,0.0
...,...,...,...,...,...
33496,흥아해운(주),3280,2010/12,0.0,0.0
33498,흥아해운(주),3280,2012/12,0.0,0.0
33499,흥아해운(주),3280,2013/12,0.0,0.0
33500,흥아해운(주),3280,2014/12,0.0,0.0


In [184]:
w = q.drop_duplicates(['회사명'])
w

,회사명,거래소코드,회계년도,부실징후기업,한계기업
0,(주)CMG제약,58820,2000/12,2.0,0.0
20,(주)DB하이텍,990,2000/12,0.0,0.0
40,(주)ES큐브,50120,2001/06,2.0,0.0
59,(주)HJ중공업,97230,2007/12,2.0,0.0
72,(주)MH에탄올,23150,2000/12,0.0,0.0
...,...,...,...,...,...
33434,휴림로봇(주),90710,2003/12,2.0,0.0
33451,휴마시스(주),205470,2014/12,2.0,0.0
33457,휴젤(주),145020,2011/12,2.0,0.0
33467,흥구석유(주),24060,2001/12,0.0,0.0


In [196]:
e = df_1[df_1['한계기업']!=1]
e = df_1[df_1['한계기업']!=0]
e.head(10)

,회사명,거래소코드,회계년도,부실징후기업,한계기업
2,(주)CMG제약,58820,2002/12,1.0,NaN
3,(주)CMG제약,58820,2003/12,1.0,NaN
4,(주)CMG제약,58820,2004/12,1.0,1.0
5,(주)CMG제약,58820,2005/12,1.0,1.0
6,(주)CMG제약,58820,2006/12,1.0,1.0
8,(주)CMG제약,58820,2008/12,1.0,NaN
9,(주)CMG제약,58820,2009/12,1.0,NaN
10,(주)CMG제약,58820,2010/12,1.0,1.0
12,(주)CMG제약,58820,2012/12,1.0,NaN
13,(주)CMG제약,58820,2013/12,1.0,NaN


In [186]:
e['한계기업'].value_counts()

1.0    3070
Name: 한계기업, dtype: int64

In [187]:
a = df_1[df_1["한계기업"]==1]
a

,회사명,거래소코드,회계년도,부실징후기업,한계기업
4,(주)CMG제약,58820,2004/12,1.0,1.0
5,(주)CMG제약,58820,2005/12,1.0,1.0
6,(주)CMG제약,58820,2006/12,1.0,1.0
10,(주)CMG제약,58820,2010/12,1.0,1.0
28,(주)DB하이텍,990,2008/12,1.0,1.0
...,...,...,...,...,...
33445,휴림로봇(주),90710,2014/12,1.0,1.0
33494,흥아해운(주),3280,2008/12,1.0,1.0
33495,흥아해운(주),3280,2009/12,1.0,1.0
33504,흥아해운(주),3280,2018/12,1.0,1.0


In [189]:
# a.to_csv('a.csv', encoding='cp949')

In [188]:
b = a.drop_duplicates(['회사명'])
b

,회사명,거래소코드,회계년도,부실징후기업,한계기업
4,(주)CMG제약,58820,2004/12,1.0,1.0
28,(주)DB하이텍,990,2008/12,1.0,1.0
46,(주)ES큐브,50120,2007/12,1.0,1.0
64,(주)HJ중공업,97230,2012/12,1.0,1.0
92,(주)SH에너지화학,2360,2000/06,1.0,1.0
...,...,...,...,...,...
33082,현대에이치티(주),39010,2011/12,1.0,1.0
33231,현우산업(주),92300,2014/12,1.0,1.0
33288,화성산업(주),2460,2011/12,1.0,1.0
33440,휴림로봇(주),90710,2009/12,1.0,1.0


In [144]:
# b.to_csv('2년연속.csv', encoding='cp949')